### LIBRARIES

In [1]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, range_boundaries
#from copy import deepcopy
import pandas as pd
import win32com.client

### BROWSING FOR PARENT FILE

In [2]:
def browse():
    global filename
    filename = filedialog.askopenfilename(initialdir = '/',
                                          title = 'Select a file',
                                          filetypes = (('Excel workbooks', '*.xlsx*'),
                                                      ('CSV files', '*.csv*'))
                                         )
    print(filename)

def close():
    window.destroy()

window = Tk()
window.title('File explorer')
window.geometry('300x200')
Label(window, text = 'Browse for the parent file').pack()
Button(window, text = 'Browse', command = browse).pack(pady = 10)
Button(window, text = 'Exit', command = close).pack(pady = 10)
window.attributes('-topmost', 1)
window.update()
window.attributes('-topmost', 0)
window.mainloop()

C:/Users/DELL/Downloads/testdataset.xlsx


### CREATING BRANCHES

OBTAINING SHEETNAMES

In [3]:
wb = load_workbook(filename)
sh_names = wb.sheetnames

TRANSFORM SHEET

In [4]:
def transform_sht(sheet):
    wb = load_workbook(filename)
    #ws = deepcopy(wb[sheet])
    ws = wb.copy_worksheet(wb[sheet])
    for group in sorted(ws.merged_cells.ranges):
        min_col, min_row, max_col, max_row = range_boundaries(str(group))
        ws.unmerge_cells(str(group))
        top_left_cell_value = ws.cell(row = min_row, column = min_col).value
        for row in ws.iter_rows(min_col = min_col, min_row = min_row, max_col = max_col, max_row = max_row):
            for cell in row:
                cell.value = top_left_cell_value    
    tbl = pd.read_excel(filename, sheet_name = sheet, skiprows = 3, header = None)
    cols = []
    for i in range(1, len(tbl.columns) + 1):
        col_letter = get_column_letter(i)
        if ws[col_letter + str(2)].value != ws[col_letter + str(3)].value:
            cols.append(str(ws[col_letter + str(2)].value or '') + '_' + str(ws[col_letter + str(3)].value or ''))
        else:
            cols.append(ws[col_letter + str(3)].value)
    tbl.columns = cols
    str_cols = list(tbl.dtypes[tbl.dtypes == 'object'].index)
    date_cols = list(tbl.dtypes[tbl.dtypes == 'datetime64[ns]'].index)
    for col in date_cols:
        tbl[col] = tbl[col].dt.strftime('%d-%m-%Y')
    uniq = {}
    for col in tbl.columns:
        uniq[col] = tbl[col].unique()
    return tbl, str_cols, uniq

FETCHING CONTACTS

In [5]:
def get_contacts(filename):
    contacts = []
    with open(filename, 'r') as f:
        for cont in f:
            contacts.append(cont.strip())
    return contacts

ADDING CONTACTS

In [6]:
def add_contact(filename, contact):
    with open(filename, 'a') as f:
        f.write(contact + '\n')

OBTAINING FILTERS

In [7]:
frames = []
filters = {} #{frame: [fr1, fr2, ...]}
sheets = {} #{frame: sheet}
tables = {} #{frame: table} 
uniqs = {} #{frame: uniq}
sel_values = {} #{fr1: [val1, val2, ...]}
cols = {} #{frame: [col1, col2, ...]}
opts = {} #{fr1: col} chosen columns
tos = {} #{frame: [to1, to2, ...]}
ccs = {} #{frame: [cc1, cc2, ...]}
contacts = get_contacts('contacts.txt') + ['Other']
#each fr corresponds to one column of the dataframe

def ui_filter(frame):
    filter_num = len(filters[frame]) + 1
    fr = Frame(frame)
    fr.pack()
    Label(fr, text = 'Filter ' + str(filter_num)).pack()
    v_opt = StringVar(fr, 'Select a column', str(fr) + '_col')
    OptionMenu(fr, v_opt, *cols[frame]).pack(pady = 10)
    Button(fr, text = 'Select this column', command = lambda: select_col(v_opt.get(), fr)).pack()
    Button(frame, text = 'Add filter', name = 'add_filter_button', command = lambda: add_filter(frame)).pack(pady = 10)
    if 'to' in frame.children:
        frame.nametowidget('to').pack()
        frame.nametowidget('cc').pack()
    filters[frame].append(fr)
    
def select_sheet(fr_sh, sheet):
    #global tbl, str_cols, uniq
    frame = fr_sh.master
    fr_sh.destroy()
    sheets[frame] = sheet
    tbl, str_cols, uniq = transform_sht(sheet)
    tables[frame] = tbl
    cols[frame] = str_cols
    uniqs[frame] = uniq
    filters[frame] = [] 
    ui_filter(frame)
    # email list
    tos[frame] = []
    ccs[frame] = []
    ui_to_cc(frame, 'to')
    ui_to_cc(frame, 'cc')
    if len(frames):
        Button(frame, text = 'Remove book', command = lambda: remove(frame)).pack(pady = 10)
    Button(window, text = 'Add book', name = 'add_button', command = add).pack(pady = 10)
    frames.append(frame)
        
def select_col(col, fr):
    #global uniq
    uniq = uniqs[fr.master]
    opts[fr] = fr.getvar(str(fr) + '_col')
    Label(fr, text = 'Select the desired value(s)').pack(pady = 10)
    for val in uniq[col]:
        chk = ttk.Checkbutton(fr, text = val)
        chk.state(['!alternate'])
        chk.state(['!selected'])
        chk.pack()
    Button(fr, text = 'Apply filter', command = lambda: apply_filter(col, fr)).pack(pady = 10)

def apply_filter(col, fr):
    #global uniq
    uniq = uniqs[fr.master]
    sel_values[fr] = []
    for i, val in enumerate(uniq[col]):
        if not i:
            st = fr.nametowidget('!checkbutton').state()
        else:
            st = fr.nametowidget('!checkbutton' + str(i + 1)).state()
        print(st)
        if len(st) and (st[0] == 'selected' or st[1] == 'selected'):
            print(val)
            sel_values[fr].append(val)
    if len(filters[fr.master]) > 1:
        Button(fr, text = 'Remove filter', command = lambda: remove_filter(col, fr)).pack(pady = 10)
    if col in cols[fr.master]:
        cols[fr.master].remove(col)
    
def remove_filter(col, fr):
    filters[fr.master].remove(fr)
    for i, f in enumerate(filters[fr.master]):
        f.nametowidget('!label').configure(text = 'Filter ' + str(i + 1))
    cols[fr.master].append(col)
    fr.destroy()
    
def add_filter(frame):
    frame.nametowidget('add_filter_button').destroy()
    frame.nametowidget('to').pack_forget()
    frame.nametowidget('cc').pack_forget()
    ui_filter(frame)

def ui_to_cc(frame, to_cc):
    if to_cc == 'to':
        text = 'To:' 
    else:
        text = 'Cc:'
    frame_to_cc = Frame(frame, name = to_cc)
    frame_to_cc.pack()
    Label(frame_to_cc, text = text).pack(pady = 10)
    v_to_cc = StringVar(frame, 'Select a contact')
    OptionMenu(frame_to_cc, v_to_cc, *contacts).pack()
    Button(frame_to_cc, text = 'Add recipient', command = lambda: add_to_cc(v_to_cc, frame_to_cc, to_cc), name = to_cc + '_button').pack(pady = 10)    
    
def ui():
    book_num = len(frames) + 1
    frame = Frame(window)
    frame.pack()
    Label(frame, text = 'Book ' + str(book_num)).pack(pady = 10)
    # selecting sheet
    v_sh = StringVar(frame, 'Select a sheet')
    fr_sh = Frame(frame)
    fr_sh.pack()
    OptionMenu(fr_sh, v_sh, *sh_names).pack(pady = 10)
    Button(fr_sh, text = 'Select sheet', command = lambda: select_sheet(fr_sh, v_sh.get())).pack()
    
def add_to_cc(v_to_cc, frame_to_cc, to_cc):
    v_to_val = v_to_cc.get()
    if v_to_val == 'Other':
        frame_to_cc.nametowidget('!optionmenu').destroy()
        frame_to_cc.nametowidget(to_cc + '_button').destroy()
        v_to_cc.set('Enter the contact email')
        Entry(frame_to_cc, textvariable = v_to_cc).pack()
        Button(frame_to_cc, text = 'Add recipient', command = lambda: add_to_cc(v_to_cc, frame_to_cc, to_cc), name = to_cc + '_button').pack(pady = 10)
    elif to_cc == 'to':
        tos[frame_to_cc.master].append(v_to_val)
        if v_to_val not in contacts:
            add_contact('contacts.txt', v_to_val)
            contacts.append(v_to_val)
    else:
        ccs[frame_to_cc.master].append(v_to_val)
        if v_to_val not in contacts:
            add_contact('contacts.txt', v_to_val)
            contacts.append(v_to_val)

def add():
    window.nametowidget('add_button').destroy()
    ui()
    
def remove(frame):
    frames.remove(frame)
    frame.destroy()
    for i, fr in enumerate(frames):
        fname = str(fr)
        fr.nametowidget('!label').configure(text = 'Book ' + str(i + 1))       
        
def adjust_scrollregion(event):
    canv.configure(scrollregion = canv.bbox('all'))
    
root = Tk()
root.title('Filters')
canv = Canvas(root)
window = Frame(canv)
scbar = Scrollbar(root)
canv.config(yscrollcommand = scbar.set)
scbar.config(orient = VERTICAL, command = canv.yview)
scbar.pack(side = RIGHT, fill = Y)
canv.pack(fill = BOTH, side = LEFT, expand = TRUE)
canv.create_window(200, 0, window = window, anchor = 'c')
ui()
window.bind('<Configure>', adjust_scrollregion)
root.attributes('-topmost', 1)
root.update()
root.attributes('-topmost', 0)
root.mainloop()

()
('selected',)
DB2
('focus', 'selected')
DB3
()
('focus', 'selected')
VA
()
()
('selected',)
MANGO
()
('focus', 'selected')
APPLE
()


HANDLING MERGED CELLS DURING ROW DELETION

In [8]:
def delete_row_with_merged_ranges(sheet, idx):
    sheet.delete_rows(idx)
    for mcr in sheet.merged_cells:
        if idx < mcr.min_row:
            mcr.shift(row_shift =- 1)
        elif idx < mcr.max_row:
            mcr.shrink(bottom = 1)

APPLYING THE FILTERS

In [9]:
for i, book in enumerate(frames):
    tbl = tables[book]
    df_filter = tbl.copy()
    wb = load_workbook(filename)
    for sheet in sh_names:
        if sheet != sheets[book]:
            del wb[sheet]
    ws = wb[sheets[book]]
    ws.delete_rows(4, len(df_filter))
    delete_row_with_merged_ranges(ws, 1)
    for fr in filters[book]:
        filter_col = opts[fr]
        val_list = sel_values[fr]
        df_filter = df_filter.loc[df_filter[filter_col].isin(val_list)]
    rows = df_filter.values.tolist()
    print(rows)
    for row in rows:
        ws.append(row)
    wb.save(sheets[book] + '.xlsx')

[['VA', 'DB3', 21, 'APPLE', 'MOK', 'FJDSNFJK', nan, '11-06-2023', 'KDNVLKD', '11-06-2023'], ['VA', 'DB2', 19, 'BERRY', 'BOK', 'DJNFJKEN', nan, '14-06-2023', 'KJDNKJDS', '14-06-2023']]
[['VA', 'DB1', 15, 'MANGO', 'TOK', 'FDSJFSDJFK', 'DJFNKJDSNF', '05-06-2023', 'NVKSDNVKSD', '05-06-2023'], ['TX', 'DB2', 15, 'MANGO', 'TOK', 'JVNJSDNV', 'DJNJKDNDJC', '06-06-2023', 'NVKD', '06-06-2023'], ['AZ', 'DB3', 15, 'MANGO', 'TOK', 'JDNJKDN', 'CV SD VKJ', '07-06-2023', 'M VDK', '07-06-2023'], ['AZ', 'DB2', 21, 'APPLE', 'MOK', 'DFJKDNF', 'DNKJDNV', '10-06-2023', 'NSVKJNSDJVND', '10-06-2023'], ['VA', 'DB3', 21, 'APPLE', 'MOK', 'FJDSNFJK', 'JNVKJDSN', '11-06-2023', 'KDNVLKD', '11-06-2023'], ['AZ', 'DB1', 21, 'APPLE', 'MOK', 'FJDSNFJK', 'NKJDSKNCD', '13-06-2023', 'VDKSDK', '13-06-2023']]


SENDING MAILS

In [10]:
for i, frame in enumerate(frames):
    ol = win32com.client.Dispatch("outlook.application") #outlook application should be installed prior
    olmailitem = 0x0 #size of the new email
    newmail = ol.CreateItem(olmailitem)
    newmail.Subject = 'Testing Mail'
    newmail.To = ';'.join(tos[frame])
    newmail.CC = ';'.join(ccs[frame])
    newmail.Body = 'Please find property comparison of this app and provide sign off.'
    newmail.Attachments.Add('C:\\Users\\zkutmyb\\' + 'Book ' + str(i + 1) + '.xlsx') #insert appropriate 'absolute' file path
    # Save mail as draft
    newmail.Display() 
    # Send mail
    #newmail.Send()

com_error: (-2147221005, 'Invalid class string', None, None)